In [2]:
import os
import pandas as pd
import numpy as np
import json

# offical_names = ['MinerU-0.9.3', 'Marker-0.2.17', 'Marker-1.2.3', 'Mathpix', 'docling', 'GOT-OCR', 'Nougat', 'GPT4o', 'Qwen2-VL-72B', 'InternVL2-Llama3-76B']
# ocr_types = ['mineru1110_docyolo', 'marker', 'marker0106', 'mathpix1108', 'docling', 'GOT', 'nougat', 'gpt4o_long_prompt', 'QwenVL_long_prompt', 'internvl_long_prompt']
ocr_types = ['mistral_ocr_results']
offical_names = ocr_types

result_folder = '../result'

# match_name = 'no_split'
match_name = 'quick_match'

ModuleNotFoundError: No module named 'pandas'

In [1]:
# overall result

dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    save_dict = {}
    en_overall = []
    ch_overall = []
    for category_type, metric in [("text_block", "Edit_dist"), ("display_formula", "Edit_dist"), ("display_formula", "CDM"), ("table", "TEDS"), ("table", "Edit_dist"), ("reading_order", "Edit_dist")]:
        if metric == 'CDM':
            save_dict[category_type+'_'+metric+'_EN'] = '-'
            save_dict[category_type+'_'+metric+'_CH'] = '-'
        elif metric == "TEDS":
            save_dict[category_type+'_'+metric+'_EN'] = result[category_type]["page"][metric]["language: english"] * 100
            save_dict[category_type+'_'+metric+'_CH'] = result[category_type]["page"][metric]["language: simplified_chinese"] * 100
            # save_dict[category_type+'_'+metric+'_EN'] = '-'
            # save_dict[category_type+'_'+metric+'_CH'] = '-'
        else:
            save_dict[category_type+'_'+metric+'_EN'] = result[category_type]["page"][metric].get("language: english", np.nan)
            save_dict[category_type+'_'+metric+'_CH'] = result[category_type]["page"][metric].get("language: simplified_chinese",np.nan)
        if metric == "Edit_dist":
            en_overall.append(result[category_type]["page"][metric].get("language: english", np.nan))
            ch_overall.append(result[category_type]["page"][metric].get("language: simplified_chinese",np.nan))
    
    save_dict['overall_EN'] = sum(en_overall) / len(en_overall)
    save_dict['overall_CH'] = sum(ch_overall) / len(ch_overall)
    dict_list.append(save_dict)
    
df = pd.DataFrame(dict_list, index=offical_names).round(3)
# df.to_csv('./overall.csv')

df

NameError: name 'ocr_types' is not defined

In [ ]:
# PDF type
dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['text_block']["page"]["Edit_dist"])
df2 = pd.DataFrame(dict_list, index=offical_names)

reordered_df2 = df2.round(3)

selected_columns = reordered_df2[["data_source: book", "data_source: PPT2PDF", "data_source: research_report", "data_source: colorful_textbook", "data_source: exam_paper", "data_source: magazine", "data_source: academic_literature", "data_source: note", "data_source: newspaper"]]
# calculate mean
selected_columns['mean'] = reordered_df2["ALL"]
# selected_columns['variance'] = selected_columns.var(axis=1)
# selected_columns.to_csv('./data_source.csv')

selected_columns

C:\Users\v-jerryfeng\AppData\Local\Temp\ipykernel_9160\492749487.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['mean'] = reordered_df2["ALL"]


,data_source: book,data_source: PPT2PDF,data_source: research_report,data_source: colorful_textbook,data_source: exam_paper,data_source: magazine,data_source: academic_literature,data_source: note,data_source: newspaper,mean
end2end,0.43,0.004,0.613,0.49,0.039,0.362,0.123,0.582,0.562,0.356


In [ ]:
# page special issue

def get_columns(df, required_columns):
    for col in required_columns:
        if col not in df.columns:
            df[col] = float('nan')

    # Then select the columns
    selected_columns = df[required_columns]
    return selected_columns

dict_list = []
dict_list_var = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['text_block']["page"]["Edit_dist"])
    dict_list_var.append(result['text_block']["page"]["Edit_dist_var"])

df2 = pd.DataFrame(dict_list, index=offical_names)
df2_var = pd.DataFrame(dict_list_var, index=offical_names)
reordered_df2 = df2.round(3)
reordered_df2_var = df2_var.round(3)

# Ensure all required columns exist, adding them with NaN values if they don't
required_columns = ["layout: single_column", "layout: double_column", "layout: three_column", "layout: other_layout"]

# Then select the columns
selected_columns = get_columns(reordered_df3, required_columns)
selected_columns_var = get_columns(reordered_df3_var, required_columns)

selected_columns.rename(columns={'fuzzy_scan':'fuzzy_scan_mean','watermark':'watermark_mean','colorful_backgroud':'colorful_backgroud_mean'},inplace=True)
selected_columns_var.rename(columns={'fuzzy_scan':'fuzzy_scan_var','watermark':'watermark_var','colorful_backgroud':'colorful_backgroud_var'},inplace=True)
result = pd.merge(selected_columns, selected_columns_var, left_index=True, right_index=True)
result = result.reindex(columns=['fuzzy_scan_mean','fuzzy_scan_var','watermark_mean','watermark_var','colorful_backgroud_mean','colorful_backgroud_var'])
result

NameError: name 'reordered_df3' is not defined

In [ ]:
# reading order under different layout
dict_list = []
dict_list_var = []
for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['reading_order']["page"]["Edit_dist"])
    dict_list_var.append(result['text_block']["page"]["Edit_dist_var"])
df3 = pd.DataFrame(dict_list, index=offical_names)
df3_var = pd.DataFrame(dict_list_var, index=offical_names)
reordered_df3 = df3.round(3)
reordered_df3_var = df3_var.round(3)


selected_columns3 = reordered_df3[["layout: single_column", "layout: double_column", "layout: three_column", "layout: other_layout"]]
selected_columns3_var = reordered_df3_var[["layout: single_column", "layout: double_column", "layout: three_column", "layout: other_layout"]]
# calculate mean and variance
selected_columns3.rename(columns={'layout: single_column':'layout: single_column_mean','layout: double_column':'layout: double_column_mean','layout: three_column':'layout: three_column_mean','layout: other_layout':'layout: other_layout_mean'},inplace=True)
selected_columns3_var.rename(columns={'layout: single_column':'layout: single_column_var','layout: double_column':'layout: double_column_var','layout: three_column':'layout: three_column_var','layout: other_layout':'layout: other_layout_var'},inplace=True)
result = pd.merge(selected_columns3, selected_columns3_var, left_index=True, right_index=True)
result = result.reindex(columns=['layout: single_column_mean','layout: single_column_var','layout: double_column_mean','layout: double_column_var','layout: three_column_mean','layout: three_column_var','layout: other_layout_mean','layout: other_layout_var'])
result

C:\Users\v-jerryfeng\AppData\Local\Temp\ipykernel_9160\2595941010.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns3.rename(columns={'layout: single_column':'layout: single_column_mean','layout: double_column':'layout: double_column_mean','layout: three_column':'layout: three_column_mean','layout: other_layout':'layout: other_layout_mean'},inplace=True)
C:\Users\v-jerryfeng\AppData\Local\Temp\ipykernel_9160\2595941010.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns3_var.rename(columns={'layout: single_column':'layout: single_column_var','layout: double_column':'layout: double_c

,layout: single_column_mean,layout: single_column_var,layout: double_column_mean,layout: double_column_var,layout: three_column_mean,layout: three_column_var,layout: other_layout_mean,layout: other_layout_var
end2end,0.084,0.095,0.299,0.137,0.0,NaN,0.6,0.134


In [ ]:
# text attribute
dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['text_block']["group"]["Edit_dist"])

df4 = pd.DataFrame(dict_list, index=offical_names)
df4 = df4.round(3)

selected_columns = df4[["text_language: text_english", "text_language: text_simplified_chinese", "text_language: text_en_ch_mixed", "text_background: white", "text_background: single_colored", "text_background: multi_colored"]]

# selected_columns.to_csv('.text_attribute.csv')
selected_columns

,text_language: text_english,text_language: text_simplified_chinese,text_language: text_en_ch_mixed,text_background: white,text_background: single_colored,text_background: multi_colored
end2end,0.093,0.735,0.318,0.531,0.341,0.36


In [ ]:
# table attribute
dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_{match_name}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['table']["group"]["TEDS"])

df4 = pd.DataFrame(dict_list, index=offical_names)
df4 = df4 * 100
df4 = df4.round(1)

selected_columns = df4[["language: table_en", "language: table_simplified_chinese", "language: table_en_ch_mixed", "line: full_line", "line: less_line", "line: fewer_line", "line: wireless_line", 
                        "with_span: True", "with_span: False", "include_equation: True", "include_equation: False", "include_background: True", "include_background: False", "table_layout: vertical", "table_layout: horizontal"]]

# selected_columns.to_csv('./table_attribute.csv')
selected_columns

In [ ]:
# text Recognition

ocr_types = ['OmniDocBench_easyocr_text_ocr', 'OmniDocBench_openocr_text_ocr']
offical_names = ocr_types

result_folder = '../result'

dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result["group"]["Edit_dist"])

df4 = pd.DataFrame(dict_list, index=offical_names)
df4 = df4.round(3)

selected_columns = df4[["text_language: text_english", "text_language: text_simplified_chinese", "text_language: text_en_ch_mixed", "text_background: white", "text_background: single_colored", "text_background: multi_colored", "text_rotate: normal", "text_rotate: rotate90", "text_rotate: rotate270", "text_rotate: horizontal"]]

# selected_columns.to_csv('.text_attribute.csv')
selected_columns

In [ ]:
# table Recognition

ocr_types = ['OmniDocBench_rapidtable_ocr']
offical_names = ocr_types

result_folder = '../result'

dict_list = []

for ocr_type in ocr_types:
    result_path = os.path.join(result_folder, f'{ocr_type}_metric_result.json')
    
    with open(result_path, 'r') as f:
        result = json.load(f)
    
    # for category_type in result.keys():
    dict_list.append(result['table']["group"]["TEDS"])

df4 = pd.DataFrame(dict_list, index=offical_names)
df4 = df4 * 100
df4 = df4.round(1)

selected_columns = df4[["language: table_en", "language: table_simplified_chinese", "language: table_en_ch_mixed", "line: full_line", "line: less_line", "line: fewer_line", "line: wireless_line", 
                        "with_span: True", "with_span: False", "include_equation: True", "include_equation: False", "include_background: True", "include_background: False", "table_layout: vertical", "table_layout: horizontal"]]

# selected_columns.to_csv('./table_attribute.csv')
selected_columns